In [53]:
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import os

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

In [58]:
file_path_train = "C:\Sagar\JaneStreetData\\train.parquet\\partition_id=0"
file_path_lags = "C:\Sagar\JaneStreetData\lags.parquet\date_id=0"

value_of_interest = 'responder_6'

train_df = pd.read_parquet(file_path_train, engine="pyarrow")
lags_df = pd.read_parquet(file_path_lags, engine="pyarrow")

columns_with_all_nan = train_df.columns[train_df.isnull().all()].tolist()
train_df = train_df.drop(columns=columns_with_all_nan)

# columns_with_nan = train_df.columns[train_df.isnull().any()].tolist()
# train_df = train_df.dropna(axis=1)
columns_to_drop = [col for col in train_df.columns if col.startswith("responder_") and col != value_of_interest]
train_df = train_df.drop(columns=columns_to_drop)
train_df.head(38)


,date_id,time_id,symbol_id,weight,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,...,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_6
0,0,0,1,3.889038,0.851033,0.242971,0.263400,-0.891687,11,7,...,-1.022690,0.152241,-0.659864,NaN,NaN,-0.261412,-0.211486,-0.335556,-0.281498,0.775981
1,0,0,7,1.370613,0.676961,0.151984,0.192465,-0.521729,11,7,...,-1.052685,-0.393726,-0.741603,NaN,NaN,-0.281207,-0.182894,-0.245565,-0.302441,0.703665
2,0,0,9,2.285698,1.056285,0.187227,0.249901,-0.773050,11,7,...,-0.863230,-0.241892,-0.709919,NaN,NaN,0.377131,0.300724,-0.106842,-0.096792,2.109352
3,0,0,10,0.690606,1.139366,0.273328,0.306549,-1.262223,42,5,...,-0.530602,4.765215,0.571554,NaN,NaN,-0.226891,-0.251412,-0.215522,-0.296244,1.114137
4,0,0,14,0.440570,0.955200,0.262404,0.344457,-0.613813,44,3,...,-1.141761,0.099631,-0.661928,NaN,NaN,3.678076,2.793581,2.618250,3.418133,-3.572820
5,0,0,16,1.118269,1.092428,0.241437,0.309494,-1.047909,11,7,...,-0.522513,-0.163335,-0.706811,NaN,NaN,-0.272425,-0.253911,-0.346411,-0.189784,0.840550
6,0,0,19,2.456331,0.747231,0.168143,0.193987,-0.602202,4,3,...,-0.990042,1.306702,-0.356026,NaN,NaN,-0.330015,-0.249928,-0.210434,-0.187166,-0.807892
7,0,0,33,1.663408,1.182569,0.206299,0.253336,-0.946356,11,7,...,-0.710232,-0.222114,-0.823108,NaN,NaN,-0.347611,-0.286403,-0.399971,-0.171215,0.965387
8,0,1,1,3.889038,0.917613,-0.401377,0.066465,-0.724457,11,7,...,-1.287999,0.196725,-0.319822,NaN,NaN,-0.211037,-0.235291,-0.269429,-0.301006,0.751976
9,0,1,7,1.370613,0.877172,-0.254713,0.047210,-0.673954,11,7,...,-0.965937,-0.192011,-0.498838,NaN,NaN,-0.293913,-0.240564,-0.208652,-0.294585,0.635277


In [57]:
lags_df

,date_id,time_id,symbol_id,responder_0_lag_1,responder_1_lag_1,responder_2_lag_1,responder_3_lag_1,responder_4_lag_1,responder_5_lag_1,responder_6_lag_1,responder_7_lag_1,responder_8_lag_1
0,0,0,0,-0.442215,-0.322407,0.143594,-0.926890,-0.782236,-0.036595,-1.305746,-0.795677,-0.143724
1,0,0,1,-0.651829,-1.707840,-0.893942,-1.065488,-1.871338,-0.615652,-1.162801,-1.205924,-1.245934
2,0,0,2,-0.656373,-0.264575,-0.892879,-1.511886,-1.033480,-0.378265,-1.574290,-1.863071,-0.027343
3,0,0,3,-0.188186,-0.190970,-0.701490,0.098453,-1.015506,-0.054984,0.329152,-0.965471,0.576635
4,0,0,4,-0.257462,-0.471325,-0.297420,0.074018,-0.324194,-0.597093,0.219856,-0.276356,-0.904790
5,0,0,5,0.027579,-0.020169,0.640348,-0.948373,-0.374251,-0.240350,-0.913801,-0.548867,-1.283726
6,0,0,6,-0.419646,-0.181228,-0.194079,0.667993,0.936857,0.517728,0.896325,1.068884,1.579290
7,0,0,7,-0.114118,-0.198511,-0.200027,-0.410021,-0.135167,-0.182887,-0.492168,-0.142915,-0.202081
8,0,0,8,-0.374147,0.092127,0.294723,0.402989,2.060188,-0.225042,0.956460,2.185598,-0.435856
9,0,0,9,-0.529529,0.040104,-0.333090,-0.959040,-1.318411,-0.774299,-0.716492,-1.471419,-1.107083


In [51]:
# Extract the first three columns for comparison
train_keys = train_df.iloc[:, :3]
lag_keys = lags_df.iloc[:, :3]

train_keys_tuples = set(train_keys.apply(tuple, axis=1))
lag_keys_tuples = set(lag_keys.apply(tuple, axis=1))

# Find matching keys
matching_keys = train_keys_tuples & lag_keys_tuples

# Filter rows in both datasets
train_filtered = train_df[train_keys.apply(tuple, axis=1).isin(matching_keys)]
lags_filtered = lags_df[lag_keys.apply(tuple, axis=1).isin(matching_keys)].iloc[:, 3:]

lags_filtered = lags_filtered.reset_index(drop=True)


# Separate the last column from the first dataset
last_column = train_filtered.iloc[:, -1]  # Extract the last column
main_data_without_last = train_filtered.iloc[:, :-1]  # All columns except the last

# Concatenate the datasets
combined_data = pd.concat([main_data_without_last, lags_filtered, last_column], axis=1)

# Rename the last column to its original name
combined_data.rename(columns={combined_data.columns[-1]: train_filtered.columns[-1]}, inplace=True)


In [52]:
combined_data

,date_id,time_id,symbol_id,weight,feature_05,feature_06,feature_07,feature_09,feature_10,feature_11,...,responder_0_lag_1,responder_1_lag_1,responder_2_lag_1,responder_3_lag_1,responder_4_lag_1,responder_5_lag_1,responder_6_lag_1,responder_7_lag_1,responder_8_lag_1,responder_6
0,0,0,1,3.889038,0.851033,0.242971,0.263400,11,7,76,...,-0.651829,-1.707840,-0.893942,-1.065488,-1.871338,-0.615652,-1.162801,-1.205924,-1.245934,0.775981
1,0,0,7,1.370613,0.676961,0.151984,0.192465,11,7,76,...,-0.114118,-0.198511,-0.200027,-0.410021,-0.135167,-0.182887,-0.492168,-0.142915,-0.202081,0.703665
2,0,0,9,2.285698,1.056285,0.187227,0.249901,11,7,76,...,-0.529529,0.040104,-0.333090,-0.959040,-1.318411,-0.774299,-0.716492,-1.471419,-1.107083,2.109352
3,0,0,10,0.690606,1.139366,0.273328,0.306549,42,5,150,...,-0.709064,-0.137431,-0.475960,-0.506644,-0.297788,-0.530738,-0.263427,-0.169489,-0.410877,1.114137
4,0,0,14,0.440570,0.955200,0.262404,0.344457,44,3,16,...,0.464961,0.077041,0.601805,-0.178444,1.127965,-0.445524,-0.507432,0.985169,-1.497043,-3.572820
5,0,0,16,1.118269,1.092428,0.241437,0.309494,11,7,76,...,0.138667,0.062221,-0.140365,-2.740061,-1.360370,-1.297678,-2.992689,-2.387136,-1.834790,0.840550
6,0,0,19,2.456331,0.747231,0.168143,0.193987,4,3,11,...,-0.120426,-0.257001,0.461233,0.334276,-0.430230,-0.053538,0.386924,-0.633187,-0.465996,-0.807892
7,0,0,33,1.663408,1.182569,0.206299,0.253336,11,7,76,...,-0.322945,-0.100357,0.044535,-0.853970,-1.932011,-1.108600,-1.377528,-2.624511,-0.798179,0.965387


In [4]:
# Path to the train.parquet directory
data_dir = "C:\Sagar\JaneStreetData\\train.parquet"
# partitions = ["partition_id=0", "partition_id=1"]  # Select manageable partitions
partitions = ["partition_id=0"]  # Select manageable partitions

# Function to load and preprocess a single partition
def load_and_preprocess_partition(partition_path):
    file_path = os.path.join(data_dir, partition_path)
    df = pd.read_parquet(file_path, engine="pyarrow", columns=[f"feature_{i:02d}" for i in range(20)] + ["responder_6"])

    return df_cleaned, columns_with_nan

# Initialize a list to collect data
data_frames = []
columns_with_missing_values = np.zeros(len(partitions))

# Load data from selected partitions
for i in range (len(partitions)):
    print(f"Loading data from: {partitions[i]}")
    df_partition , columns_with_missing_values= load_and_preprocess_partition(partitions[i])
    data_frames.append(df_partition)
    columns_with_missing_values[i]

print(columns_with_missing_values)
# Combine all loaded data into one DataFrame
train_df = pd.concat(data_frames, ignore_index=True)


#file_name = "example_dataframe.csv"

# Save the DataFrame to a CSV file
#data.to_csv(file_name, index=False)

Loading data from: partition_id=0
['feature_00', 'feature_01', 'feature_02', 'feature_03', 'feature_04', 'feature_08', 'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19']


In [5]:
# Example: read one partition of train data (the competition provides 10 partitions)
print(train_df.dtypes)

feature_05     float32
feature_06     float32
feature_07     float32
feature_09        int8
feature_10        int8
feature_11       int16
feature_12     float32
feature_13     float32
feature_14     float32
responder_6    float32
dtype: object


In [ ]:
# Read lags
lags_df = pd.read_parquet("C:\Sagar\JaneStreetData\lags.parquet\date_id=0", engine='pyarrow')
# Merge to bring the lagged responders for the *previous* date
# We merge on [date_id, symbol_id], but we have to shift date_id by 1 in lags
lags_df = lags_df.rename(columns={col: col + "_lag_1" for col in ["responder_0","responder_1","responder_2",
                                                                "responder_3","responder_4","responder_5",
                                                                "responder_6","responder_7","responder_8"]})
# In lags_df, we have something like date_id_lag, symbol_id, responder_0_lag, etc.
# Typically, you'd do "train_df['date_id'] - 1" to match lags. But the exact logic
# may differ if date_id’s are not strictly daily increments.

lags_df["date_id"] = lags_df["date_id"] + 1  # so it lines up with the next day in train_df

# Merge so that for each (date_id, symbol_id), we attach the previous day’s lagged responders
train_df = pd.merge(
    train_df,
    lags_df.drop_duplicates(["date_id", "symbol_id"]),  # drop duplicates if they exist
    on=["date_id", "symbol_id"],
    how="left"
)


In [ ]:
# Keep only first 50,000 rows to avoid memory issues in an example
train_df = train_df.head(50000).copy()

# Sort the data by date_id and time_id so sequences are in correct chronological order
train_df = train_df.sort_values(by=["date_id", "time_id_x"]).reset_index(drop=True)


## 4. Preparing Sequences for an RNN
We need to form sequences: for each row 𝑡 we’ll look back \text{seq_length} timesteps to build an input sequence of features, and we’ll predict the next value(s) of 
responder
6
responder 
6
​
 .

Because the dataset has multiple symbols, you have a few approaches:

Per-Symbol RNN: You train a separate model or have separate sequences for each symbol_id.
Global Model: You mix all symbols in one model, possibly embedding symbol_id.
Below, we do a simplified approach that filters to one symbol to illustrate. In practice, you’d handle each symbol or create a global multi-symbol model with an embedding for symbol_id.

In [ ]:
# Example: filter to a single symbol for demonstration (NOT for production!)
unique_symbols = train_df["symbol_id"].unique()
symbol_to_model = unique_symbols[0]
train_symbol_df = train_df[train_df["symbol_id"] == symbol_to_model].reset_index(drop=True)

# List of feature columns we want to use
feature_cols = [f"feature_{i:02d}" for i in range(79)]
# Optionally add the lagged responders as features
lag_cols = [f"responder_{i}_lag_1_y" for i in range(9)]

# Target column
target_col = "responder_6"

#### Custom Dataset
We create a Dataset that returns (X, y) pairs, where X is a sequence of length seq_length of features, and y is the target at the last timestep. (You could also predict multiple steps ahead, etc.)

In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, df, feature_cols, target_col, seq_length=16):
        """
        df: DataFrame containing sorted time-series data for a single symbol.
        feature_cols: columns used as input features
        target_col: column to predict
        seq_length: length of look-back window
        """
        self.df = df
        self.feature_cols = feature_cols
        self.target_col = target_col
        self.seq_length = seq_length
        
        # Convert relevant columns to numpy arrays for speed
        self.features = df[feature_cols].values
        self.targets = df[target_col].values
        
    def __len__(self):
        # We can only start from seq_length-1
        return len(self.df) - self.seq_length
    
    def __getitem__(self, idx):
        # Sequence of features from idx -> idx+seq_length
        x_seq = self.features[idx : idx + self.seq_length, :]
        # The target is the last row's responder_6
        y = self.targets[idx + self.seq_length - 1]
        
        # Convert to torch tensors
        x_seq = torch.tensor(x_seq, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        
        return x_seq, y


#### Create Training/Validation Splits
We might do a time-based split. For example, use the first 80% of this single-symbol data for training, last 20% for validation. (In multi-day data, you’d typically split by date.)

In [ ]:
train_df

In [ ]:
train_size = int(len(train_symbol_df) * 0.8)
train_df_ = train_symbol_df.iloc[:train_size].reset_index(drop=True)
val_df_   = train_symbol_df.iloc[train_size:].reset_index(drop=True)

SEQ_LENGTH = 16

train_dataset = TimeSeriesDataset(train_df_, feature_cols + lag_cols, target_col, seq_length=SEQ_LENGTH)
val_dataset   = TimeSeriesDataset(val_df_,   feature_cols + lag_cols, target_col, seq_length=SEQ_LENGTH)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
val_loader   = DataLoader(val_dataset,   batch_size=64, shuffle=False, drop_last=True)


#### 5. Defining the LSTM Model
We’ll define a simple LSTM-based regressor. You can of course expand this with more layers, dropout, attention, etc.

In [ ]:
class LSTMRegressor(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=1):
        """
        input_size: number of input features per timestep
        hidden_size: number of hidden units in the LSTM
        num_layers: number of stacked LSTM layers
        """
        super(LSTMRegressor, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Final linear layer to map hidden state -> 1 output (responder_6)
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        """
        x: (batch, seq_length, input_size)
        """
        # h_0, c_0 default to zeros if not provided
        out, (h_n, c_n) = self.lstm(x)  # out: (batch, seq_length, hidden_size)
        
        # We can take the last timestep's output for regression
        out = out[:, -1, :]  # (batch, hidden_size)
        
        # Map to 1 dimension
        out = self.fc(out)   # (batch, 1)
        
        return out.squeeze(-1)  # (batch,)


In [ ]:
input_size = len(feature_cols + lag_cols)  # 79 features + 9 lagged = 88
model = LSTMRegressor(input_size=input_size, hidden_size=64, num_layers=1)

# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

6. Training Loop
A simple example training loop with mean squared error (MSE) loss. In practice, you might:

Use a custom loss that aligns better with your competition metric (R^2).
Use early stopping or more advanced training heuristics.
Need distributed training or GPU usage for large data.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

NUM_EPOCHS = 3  # Increase for real training

for epoch in range(NUM_EPOCHS):
    # Training
    model.train()
    train_loss = 0.0
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        
        optimizer.zero_grad()
        preds = model(x_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * x_batch.size(0)
    
    train_loss /= len(train_loader.dataset)
    
    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x_val, y_val in val_loader:
            x_val = x_val.to(device)
            y_val = y_val.to(device)
            
            val_preds = model(x_val)
            loss_val = criterion(val_preds, y_val)
            val_loss += loss_val.item() * x_val.size(0)
    
    val_loss /= len(val_loader.dataset)
    
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}]  Train Loss: {train_loss:.4f}  Val Loss: {val_loss:.4f}")


7. (Optional) Making Predictions
Assuming you have a test DataFrame (or you are using the provided evaluation API), the typical approach to generate predictions at each timestep is:

In [ ]:
test_df = pd.read_parquet(r"C:\Sagar\JaneStreetData\test.parquet\date_id=0\part-0.parquet", engine='pyarrow')
test_df = pd.merge(
    test_df,
    lags_df.drop_duplicates(["date_id", "symbol_id"]),  # drop duplicates if they exist
    on=["date_id", "symbol_id"],
    how="left"
)
test_df.rename(columns=lambda col: col + "_y" if col.endswith("_lag_1") else col, inplace=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None):
    
    print(test_df.dtypes)
    print(train_df.dtypes)

In [ ]:
# Suppose you have a new DataFrame 'test_df' with the same features + lag columns
model.eval()
test_sequences = []  # store predictions

with torch.no_grad():
    # Transform test_df rows into sequences the same way as train
    # For real usage, you’d do it one step at a time or date/time id at a time
    # if the evaluation API calls your script repeatedly.
    
    # Example with a small dataset:
    test_data = TimeSeriesDataset(test_df, feature_cols + lag_cols, target_col, seq_length=SEQ_LENGTH)
    test_loader = DataLoader(test_data, batch_size=64, shuffle=False)
    
    predictions = []
    for x_batch, _ in test_loader:
        x_batch = x_batch.to(device)
        preds = model(x_batch)
        predictions.extend(preds.cpu().numpy().tolist())

# 'predictions' now contains the model outputs for each sequence
